In [1]:
import torch
from torchvision import datasets, transforms
transform = transforms.ToTensor()
mnist_test = datasets.MNIST(root='../data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=1, shuffle=False)

In [2]:
import sys
sys.path.append('../src')
from mnist_model import MnistCNN
# Instantiate model
model = MnistCNN()
print(model)

MnistCNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1568, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [3]:
results = []
for idx, (img, label) in enumerate(test_loader):
    output = model(img)
    pred = torch.argmax(output, dim=1).item()
    results.append({
        "index": idx,
        "input": img.squeeze().numpy().tolist(),
        "logits": output.squeeze().detach().numpy().tolist(),
        "predicted": pred,
        "label": label.item()
    })
    # For brevity, limit to first 100 examples
    if idx >= 99:
        break

import json
with open("reference_inference_results.json", "w") as f:
    json.dump(results, f)

In [ ]:
# Assume FPGA results are saved to 'fpga_predictions.txt'
with open("fpga_predictions.txt") as f:
    fpga_preds = [int(line.strip()) for line in f.readlines()]

with open("reference_inference_results.json") as f:
    ref_results = json.load(f)

correct = 0
for i, ref in enumerate(ref_results):
    py_pred = ref["predicted"]
    fpga_pred = fpga_preds[i]
    if py_pred == fpga_pred:
        correct += 1
    else:
        print(f"Mismatch at {i}: Python={py_pred}, FPGA={fpga_pred}, Label={ref['label']}")

accuracy = correct / len(ref_results)
print(f"FPGA vs Python Accuracy: {accuracy:.4f}")
